In [ ]:
!pip install geopandas

     |████████████████████████████████| 994 kB 5.3 MB/s 
     |████████████████████████████████| 15.4 MB 33 kB/s 
     |████████████████████████████████| 6.6 MB 46.6 MB/s 


In [ ]:
!sudo apt install libspatialindex-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-dev is already the newest version (1.8.5-5).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [ ]:
!sudo pip3 install rtree

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas.tools import geocode
import folium
from folium import Marker

In [ ]:
resultado = geocode("The Great Pyramid of Giza", provider="nominatim")
print(resultado)

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


                    geometry                                            address
0  POINT (31.13422 29.97916)  هرم خوفو, شارع ابو الهول السياحي, نزلة البطران...


In [ ]:
point = resultado.geometry.iloc[0]
print('Latitude: ', point.y)
print('Longitude: ', point.x)

Latitude:  29.97915995
Longitude:  31.134215650388754


In [ ]:
universidade = pd.read_csv("/content/top_universities.csv")
universidade.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


In [ ]:
universidade.shape

(100, 1)

In [ ]:
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None

universidade[['Latitude', 'Longitude', 'geometry']] = universidade.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universidade["Latitude"])) / len(universidade)) * 100))

universidade = universidade.loc[~np.isnan(universidade["Latitude"])]
universidade = gpd.GeoDataFrame(universidade, geometry=universidade.geometry)
universidade.crs = {'init': 'epsg:4326'}
universidade.head()

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


95.0% of addresses were geocoded!


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,Name,Latitude,Longitude,geometry
0,University of Oxford,51.758708,-1.255668,POINT (-1.25567 51.75871)
1,University of Cambridge,52.199852,0.119739,POINT (0.11974 52.19985)
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896)
3,ETH Zurich,47.376504,8.547321,POINT (8.54732 47.37650)
4,UCL,51.523581,-0.132977,POINT (-0.13298 51.52358)


In [ ]:
mapa = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

for idx, row in universidade.iterrows():
  Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(mapa)

mapa

In [ ]:
mundo = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europa = mundo.loc[mundo.continent=='Europe'].reset_index(drop=True)

europa_stats = europa[["name", "pop_est", "gdp_md_est"]]
europa_boundaries = europa[["name","geometry"]]

europa_boundaries.head()

,name,geometry
0,Russia,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516..."
1,Norway,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ..."
2,France,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ..."
3,Sweden,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300..."
4,Belarus,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372..."


In [ ]:
europa_stats.head()

,name,pop_est,gdp_md_est
0,Russia,142257519,3745000.0
1,Norway,5320045,364700.0
2,France,67106161,2699000.0
3,Sweden,9960487,498100.0
4,Belarus,9549747,165400.0


In [ ]:
europa = europa_boundaries.merge(europa_stats, on="name")
europa.head()

,name,geometry,pop_est,gdp_md_est
0,Russia,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516...",142257519,3745000.0
1,Norway,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ...",5320045,364700.0
2,France,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ...",67106161,2699000.0
3,Sweden,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300...",9960487,498100.0
4,Belarus,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372...",9549747,165400.0


In [ ]:
universidade_europa = gpd.sjoin(universidade, europa)

print("Nós localizamos {} universidade".format(len(universidade)))
print("Só {} universidade estão na europa (apenas em {} diferentes países)".format(len(universidade_europa), len(universidade_europa.name.unique())))

universidade_europa.head()